In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('cleaned_data.csv')

In [3]:
df.head()

,date,price,address,bedrooms,bathrooms,parking,area,median_price,population,average_age,zipcode,state
0,29 04 24,480000,"1307/60 A'beckett Street, Melbourne",2,1,0,53.0,5501000,47279,0,3000,Victoria
1,27 03 24,815000,"1103/108 Flinders Street, Melbourne",2,2,1,93.0,5501000,47279,0,3000,Victoria
2,29 02 24,3350000,"3801/35 Spring Street, Melbourne",3,2,2,175.0,10721000000,47279,0,3000,Victoria
3,29 02 24,3350000,"3801/35 Spring Street, Melbourne",3,2,2,175.0,10721000000,47279,0,3000,Victoria
4,8 12 23,590000,"508/181 Exhibition Street, Melbourne",2,1,1,80.0,5501000,47279,0,3000,Victoria


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54851 entries, 0 to 54850
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          54851 non-null  object 
 1   price         54851 non-null  int64  
 2   address       54851 non-null  object 
 3   bedrooms      54851 non-null  int64  
 4   bathrooms     54851 non-null  int64  
 5   parking       54851 non-null  int64  
 6   area          54851 non-null  float64
 7   median_price  54851 non-null  int64  
 8   population    54851 non-null  int64  
 9   average_age   54851 non-null  int64  
 10  zipcode       54851 non-null  int64  
 11  state         54851 non-null  object 
dtypes: float64(1), int64(8), object(3)
memory usage: 5.0+ MB


In [5]:
df.corr()

/tmp/ipykernel_6974/1134722465.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,price,bedrooms,bathrooms,parking,area,median_price,population,average_age,zipcode
price,1.000000,0.309295,0.426080,0.011766,0.002356,0.558588,0.131060,-0.081190,-0.299358
bedrooms,0.309295,1.000000,0.507762,0.035953,0.017322,0.248636,0.032390,0.134987,0.007798
bathrooms,0.426080,0.507762,1.000000,0.019326,0.012059,0.258647,0.045800,0.023539,-0.111766
parking,0.011766,0.035953,0.019326,1.000000,0.002687,0.003022,-0.008667,0.004558,0.019774
area,0.002356,0.017322,0.012059,0.002687,1.000000,-0.007693,-0.005291,0.010960,0.002740
median_price,0.558588,0.248636,0.258647,0.003022,-0.007693,1.000000,0.095193,-0.112824,-0.261424
population,0.131060,0.032390,0.045800,-0.008667,-0.005291,0.095193,1.000000,-0.120237,-0.375967
average_age,-0.081190,0.134987,0.023539,0.004558,0.010960,-0.112824,-0.120237,1.000000,0.102529
zipcode,-0.299358,0.007798,-0.111766,0.019774,0.002740,-0.261424,-0.375967,0.102529,1.000000


In [6]:
df['state'].value_counts()

New South Wales        17092
Victoria               11359
 Western Australia      9228
Queensland              7642
Tasmania                5689
 Southern Australia     2548
 Northern Territory     1293
Name: state, dtype: int64

In [7]:
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder

df['state'] = df['state'].str.strip()

categorical_columns = df.select_dtypes(include=['object']).columns
categorical_columns = categorical_columns.drop('state')

df_encoded = df.copy()
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df[col])
    label_encoders[col] = le


X = df_encoded.drop(columns=['state'])
y = df_encoded['state']

le_state = LabelEncoder()
le_state.fit(df['state'])
y_encoded = le_state.transform(df['state'])

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y_encoded)

y_resampled_decoded = le_state.inverse_transform(y_resampled)

resampled_df = pd.DataFrame(X_resampled, columns=X.columns)
resampled_df['state'] = y_resampled_decoded

df = resampled_df

df['state'].value_counts()

Victoria              17092
New South Wales       17092
Queensland            17092
Western Australia     17092
Southern Australia    17092
Northern Territory    17092
Tasmania              17092
Name: state, dtype: int64

In [8]:
df.corr()

/tmp/ipykernel_6974/1134722465.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,date,price,address,bedrooms,bathrooms,parking,area,median_price,population,average_age,zipcode
date,1.000000,-0.052287,-0.013144,0.037295,-0.036543,0.009777,-0.001364,-0.014609,-0.086568,0.043985,0.115561
price,-0.052287,1.000000,0.020307,0.328329,0.410552,0.009306,0.000323,0.571305,0.189383,-0.058582,-0.137202
address,-0.013144,0.020307,1.000000,-0.000812,0.007525,-0.011503,0.010709,0.022201,-0.008031,0.008878,0.000510
bedrooms,0.037295,0.328329,-0.000812,1.000000,0.467312,0.030512,0.013475,0.269440,0.034470,0.121062,0.078223
bathrooms,-0.036543,0.410552,0.007525,0.467312,1.000000,0.013259,0.003560,0.250079,0.062347,-0.018437,-0.134573
parking,0.009777,0.009306,-0.011503,0.030512,0.013259,1.000000,0.001569,0.001596,-0.010878,0.002322,0.013575
area,-0.001364,0.000323,0.010709,0.013475,0.003560,0.001569,1.000000,-0.006863,-0.011108,0.003785,-0.008626
median_price,-0.014609,0.571305,0.022201,0.269440,0.250079,0.001596,-0.006863,1.000000,0.150675,-0.054332,-0.122911
population,-0.086568,0.189383,-0.008031,0.034470,0.062347,-0.010878,-0.011108,0.150675,1.000000,-0.117117,-0.178573
average_age,0.043985,-0.058582,0.008878,0.121062,-0.018437,0.002322,0.003785,-0.054332,-0.117117,1.000000,0.290679


In [9]:
df

,date,price,address,bedrooms,bathrooms,parking,area,median_price,population,average_age,zipcode,state
0,4881,480000,7286,2,1,0,53.000000,5501000,47279,0,3000,Victoria
1,4586,815000,4610,2,2,1,93.000000,5501000,47279,0,3000,Victoria
2,4857,3350000,26527,3,2,2,175.000000,10721000000,47279,0,3000,Victoria
3,4857,3350000,26527,3,2,2,175.000000,10721000000,47279,0,3000,Victoria
4,771,590000,32788,2,1,1,80.000000,5501000,47279,0,3000,Victoria
...,...,...,...,...,...,...,...,...,...,...,...,...
119639,2513,311483,3149,1,1,1,259.033661,3271225,10397,0,6140,Western Australia
119640,5436,564462,28615,4,2,2,756.364736,5101000,20294,0,6110,Western Australia
119641,3482,415619,39765,4,1,1,1012.000000,4991000,2099,0,6430,Western Australia
119642,3188,342638,20651,3,1,1,759.552000,3951000,13625,0,6112,Western Australia


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119644 entries, 0 to 119643
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   date          119644 non-null  int64  
 1   price         119644 non-null  int64  
 2   address       119644 non-null  int64  
 3   bedrooms      119644 non-null  int64  
 4   bathrooms     119644 non-null  int64  
 5   parking       119644 non-null  int64  
 6   area          119644 non-null  float64
 7   median_price  119644 non-null  int64  
 8   population    119644 non-null  int64  
 9   average_age   119644 non-null  int64  
 10  zipcode       119644 non-null  int64  
 11  state         119644 non-null  object 
dtypes: float64(1), int64(10), object(1)
memory usage: 11.0+ MB


In [11]:
df

,date,price,address,bedrooms,bathrooms,parking,area,median_price,population,average_age,zipcode,state
0,4881,480000,7286,2,1,0,53.000000,5501000,47279,0,3000,Victoria
1,4586,815000,4610,2,2,1,93.000000,5501000,47279,0,3000,Victoria
2,4857,3350000,26527,3,2,2,175.000000,10721000000,47279,0,3000,Victoria
3,4857,3350000,26527,3,2,2,175.000000,10721000000,47279,0,3000,Victoria
4,771,590000,32788,2,1,1,80.000000,5501000,47279,0,3000,Victoria
...,...,...,...,...,...,...,...,...,...,...,...,...
119639,2513,311483,3149,1,1,1,259.033661,3271225,10397,0,6140,Western Australia
119640,5436,564462,28615,4,2,2,756.364736,5101000,20294,0,6110,Western Australia
119641,3482,415619,39765,4,1,1,1012.000000,4991000,2099,0,6430,Western Australia
119642,3188,342638,20651,3,1,1,759.552000,3951000,13625,0,6112,Western Australia


In [12]:
le_state = LabelEncoder()

In [13]:
df['state'] = le_state.fit_transform(df['state'])

In [14]:
le_state.classes_

array(['New South Wales', 'Northern Territory', 'Queensland',
       'Southern Australia', 'Tasmania', 'Victoria', 'Western Australia'],
      dtype=object)

In [15]:
df.head()

,date,price,address,bedrooms,bathrooms,parking,area,median_price,population,average_age,zipcode,state
0,4881,480000,7286,2,1,0,53.0,5501000,47279,0,3000,5
1,4586,815000,4610,2,2,1,93.0,5501000,47279,0,3000,5
2,4857,3350000,26527,3,2,2,175.0,10721000000,47279,0,3000,5
3,4857,3350000,26527,3,2,2,175.0,10721000000,47279,0,3000,5
4,771,590000,32788,2,1,1,80.0,5501000,47279,0,3000,5


In [16]:
from sklearn.model_selection import train_test_split

x = df[['bedrooms', 'bathrooms', 'parking', 'area', 'state']]
y = df['price']

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=2345)

In [17]:
from sklearn.ensemble import RandomForestRegressor
model_rf=RandomForestRegressor()
model_rf.fit(x_train,y_train)

RandomForestRegressor()

In [18]:
from sklearn.metrics import r2_score
r2=r2_score(y_test,model_rf.predict(x_test))
r2

0.4159769453171879

## hyperparameter tuning is using Bayesian Optimization

In [20]:
import optuna
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

# Define the objective function for Optuna
def objective(trial):
    # Suggest values for the hyperparameters
    param = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('depth', 4, 12),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1, 10),
        'bagging_temperature': trial.suggest_loguniform('bagging_temperature', 0.1, 10),
        'random_strength': trial.suggest_loguniform('random_strength', 0.1, 10),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
    }

    # Split the training data into training and validation sets
    x_train_split, x_val, y_train_split, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
    
    # Initialize the CatBoostRegressor with the current set of hyperparameters
    model = CatBoostRegressor(**param, verbose=0)

    # Train the model
    model.fit(x_train_split, y_train_split, eval_set=(x_val, y_val), early_stopping_rounds=50)

    # Predict and calculate the R² score
    preds = model.predict(x_val)
    r2 = r2_score(y_val, preds)

    return -r2  # We negate R² since Optuna minimizes the objective


In [21]:
# Create a study object and specify the direction as 'minimize'
study = optuna.create_study(direction='minimize')

# Optimize the study
study.optimize(objective, n_trials=50)

# Get the best hyperparameters
best_params = study.best_params
print("Best hyperparameters found: ", best_params)


[I 2024-08-06 14:48:32,827] A new study created in memory with name: no-name-fc1071c0-6c5e-4dce-b65b-aabea2a2cc99
/tmp/ipykernel_6974/728862885.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
/tmp/ipykernel_6974/728862885.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1, 10),
/tmp/ipykernel_6974/728862885.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'bagging_temperature

KeyboardInterrupt: 

In [ ]:
# Initialize the model with the best hyperparameters
best_model = CatBoostRegressor(**best_params, verbose=0)

# Train the model on the full training set
best_model.fit(x_train, y_train)

# Evaluate the model on the test set
r2 = r2_score(y_test, best_model.predict(x_test))
print("Improved R² score with Bayesian Optimization: ", r2)
